In [0]:
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
import pandas as pd

In [0]:
data = []
newdata = [line.split(',') for line in open("NBA-PBP_2016-2017.csv").readlines()]
data.extend(newdata[1:])
newdata = [line.split(',') for line in open("NBA-PBP-2017-2018.csv").readlines()]
data.extend(newdata[1:])
newdata = [line.split(',') for line in open("NBA-PBP-2018-2019.csv").readlines()]
data.extend(newdata[1:])
newdata = [line.split(',') for line in open("NBA-PBP-2019-2020.csv").readlines()]
data.extend(newdata[1:])

team = open("team.txt").read().split(', ')

Go through each line. Check if there is a new stat in its repective column. Check which team got stat and give them a point. Check for end of 3rd quarter to reset for next game

In [0]:
labels = ['GameType', 'Score_diff', 'Opponent', 'Assist_diff', 'Block_diff', 'Steal_diff', 'Off_rebound_diff', 'Def_rebound_diff', 'Foul_diff', 'Free_throw_made_diff', 'Free_throw_miss_diff', '2_make_diff', '2_miss_diff', '3_make_diff', '3_miss_diff']
winner = []
sortedData = []
temp = [0]*len(labels)

for line in data:
  if( line[6] != '1' and line[6] != '2' and line[6] != '3' ):
    continue
  
  if line[9] == 'End of 3rd quarter':
    if line[5] == line[11]:
      winner.append(1)
    else:
      winner.append(0)
    if line[1] == 'regular':
      temp[0] = 1
    temp[1] = int(line[13]) - int(line[10])
    temp[2] = team.index(line[8])

    sortedData.append(temp)
    temp = [0]*len(labels)
  
  elif line[18] != '':
    if line[12] != '':
      temp[3] += 1
    else:
      temp[3] -= 1
  
  elif line[21] != '':
    if line[12] == '':
      temp[8] += 1
    else:
      temp[8] -= 1
  
  elif line[19] != '':
    if line[12] == '':
      temp[4] += 1
    else:
      temp[4] -= 1
  
  elif line[35] != '':
    if line[12] != '':
      temp[5] += 1
    else:
      temp[5] -= 1
  
  elif line[24] == 'offensive':
    if line[12] != '':
      temp[6] += 1
    else:
      temp[6] -= 1

  elif line[24] == 'defensive':
    if line[12] != '':
      temp[7] += 1
    else:
      temp[7] -= 1

  elif line[29] == 'make':
    if line[12] != '':
      temp[9] += 1
    else:
      temp[9] -= 1

  elif line[29] == 'miss':
    if line[12] != '':
      temp[10] += 1
    else:
      temp[10] -= 1

  if line[16] == 'make':
    if line[12] != '':
      if line[15][0] == '2':
        temp[11] += 1
      else:
        temp[13] += 1
    else:
      if line[15][0] == '2':
        temp[11] -= 1
      else:
        temp[13] -= 1

  elif line[16] == 'miss':
    if line[12] != '':
      if line[15][0] == '2':
        temp[12] += 1
      else:
        temp[14] += 1
    else:
      if line[15][0] == '2':
        temp[12] -= 1
      else:
        temp[14] -= 1


Small tests

In [75]:
print(len(data))
print("%.3f%%" % (sum(winner)*100/len(winner)))

2299300
57.809%


Test using all of the collected data

In [67]:
svd = TruncatedSVD(n_components=4, n_iter=7, random_state=598)
svddata = svd.fit_transform(sortedData)

X_train, X_test, y_train, y_test = train_test_split(svddata, winner, test_size=0.33, random_state=598)

lr = LogisticRegression(solver ='lbfgs',max_iter=100, random_state=598).fit(X_train, y_train)
print("Logistic Regression: %.3f%%" % (lr.score(X_test, y_test)*100))

neigh = KNeighborsClassifier(n_neighbors=58).fit(X_train, y_train)
print("KNN: %.3f%%" % (neigh.score(X_test, y_test)*100))


X_train, X_test, y_train, y_test = train_test_split(sortedData, winner, test_size=0.33, random_state=598)

tree = DecisionTreeClassifier(max_depth=3, random_state=598).fit(X_train, y_train)
print("Decision Trees: %.3f%%" % (tree.score(X_test, y_test)*100))

forest = RandomForestClassifier(max_depth=9, random_state=598).fit(X_train, y_train)
print("Random Forest: %.3f%%" % (forest.score(X_test, y_test)*100))


kmeans = KMeans(n_clusters=2, random_state=598).fit(svddata)
print("\nKmeans:\n" + classification_report(kmeans.labels_,winner))



Logistic Regression: 83.323%
KNN: 83.020%
Decision Trees: 82.899%
Random Forest: 83.323%

Kmeans:
              precision    recall  f1-score   support

           0       0.87      0.70      0.78      2607
           1       0.73      0.88      0.80      2387

    accuracy                           0.79      4994
   macro avg       0.80      0.79      0.79      4994
weighted avg       0.80      0.79      0.79      4994



Test using just the difference in scores

In [68]:
lessData = pd.DataFrame(sortedData, columns=labels)
lessData = lessData.drop(columns=['GameType', 'Opponent', 'Assist_diff', 'Block_diff', 'Steal_diff', 'Off_rebound_diff', 'Def_rebound_diff', 'Foul_diff', 'Free_throw_made_diff', 'Free_throw_miss_diff', '2_make_diff', '2_miss_diff', '3_make_diff', '3_miss_diff'])
# svd = TruncatedSVD(n_components=4, n_iter=7, random_state=598)
# lesssvddata = svd.fit_transform(lessData)

X_train, X_test, y_train, y_test = train_test_split(lessData, winner, test_size=0.33, random_state=598)

lr = LogisticRegression(solver ='lbfgs',max_iter=100, random_state=598).fit(X_train, y_train)
print("Logistic Regression: %.3f%%" % (lr.score(X_test, y_test)*100))

neigh = KNeighborsClassifier(n_neighbors=58).fit(X_train, y_train)
print("KNN: %.3f%%" % (neigh.score(X_test, y_test)*100))


# X_train, X_test, y_train, y_test = train_test_split(lessData, winner, test_size=0.33, random_state=598)

tree = DecisionTreeClassifier(max_depth=3, random_state=598).fit(X_train, y_train)
print("Decision Trees: %.3f%%" % (tree.score(X_test, y_test)*100))

forest = RandomForestClassifier(max_depth=9, random_state=598).fit(X_train, y_train)
print("Random Forest: %.3f%%" % (forest.score(X_test, y_test)*100))


kmeans = KMeans(n_clusters=2, random_state=598).fit(lesssvddata)
print("\nKmeans:\n" + classification_report(kmeans.labels_,winner))

Logistic Regression: 82.838%
KNN: 82.838%
Decision Trees: 82.717%
Random Forest: 82.838%

Kmeans:
              precision    recall  f1-score   support

           0       0.28      0.24      0.26      2542
           1       0.33      0.39      0.35      2452

    accuracy                           0.31      4994
   macro avg       0.31      0.31      0.31      4994
weighted avg       0.31      0.31      0.31      4994



Test using all the data but score difference

In [69]:
lessData = pd.DataFrame(sortedData, columns=labels)
lessData = lessData.drop(columns=['Score_diff'])
svd = TruncatedSVD(n_components=4, n_iter=7, random_state=598)
lesssvddata = svd.fit_transform(lessData)

X_train, X_test, y_train, y_test = train_test_split(lesssvddata, winner, test_size=0.33, random_state=598)

lr = LogisticRegression(solver ='lbfgs',max_iter=100, random_state=598).fit(X_train, y_train)
print("Logistic Regression: %.3f%%" % (lr.score(X_test, y_test)*100))

neigh = KNeighborsClassifier(n_neighbors=58).fit(X_train, y_train)
print("KNN: %.3f%%" % (neigh.score(X_test, y_test)*100))


X_train, X_test, y_train, y_test = train_test_split(lessData, winner, test_size=0.33, random_state=598)

tree = DecisionTreeClassifier(max_depth=3, random_state=598).fit(X_train, y_train)
print("Decision Trees: %.3f%%" % (tree.score(X_test, y_test)*100))

forest = RandomForestClassifier(max_depth=9, random_state=598).fit(X_train, y_train)
print("Random Forest: %.3f%%" % (forest.score(X_test, y_test)*100))


kmeans = KMeans(n_clusters=2, random_state=598).fit(lesssvddata)
print("\nKmeans:\n" + classification_report(kmeans.labels_,winner))

Logistic Regression: 73.984%
KNN: 73.802%
Decision Trees: 73.681%
Random Forest: 79.442%

Kmeans:
              precision    recall  f1-score   support

           0       0.72      0.61      0.66      2487
           1       0.66      0.77      0.71      2507

    accuracy                           0.69      4994
   macro avg       0.69      0.69      0.69      4994
weighted avg       0.69      0.69      0.69      4994



Extra Interesting Tests

In [70]:
lessData = pd.DataFrame(sortedData, columns=labels)
lessData = lessData.drop(columns=['Assist_diff', 'Free_throw_made_diff', 'Free_throw_miss_diff', '2_make_diff', '2_miss_diff', '3_make_diff', '3_miss_diff'])
svd = TruncatedSVD(n_components=4, n_iter=7, random_state=598)
lesssvddata = svd.fit_transform(lessData)

X_train, X_test, y_train, y_test = train_test_split(lesssvddata, winner, test_size=0.33, random_state=598)

lr = LogisticRegression(solver ='lbfgs',max_iter=100, random_state=598).fit(X_train, y_train)
print("Logistic Regression: %.3f%%" % (lr.score(X_test, y_test)*100))

neigh = KNeighborsClassifier(n_neighbors=58).fit(X_train, y_train)
print("KNN: %.3f%%" % (neigh.score(X_test, y_test)*100))


X_train, X_test, y_train, y_test = train_test_split(lessData, winner, test_size=0.33, random_state=598)

tree = DecisionTreeClassifier(max_depth=3, random_state=598).fit(X_train, y_train)
print("Decision Trees: %.3f%%" % (tree.score(X_test, y_test)*100))

forest = RandomForestClassifier(max_depth=9, random_state=598).fit(X_train, y_train)
print("Random Forest: %.3f%%" % (forest.score(X_test, y_test)*100))


kmeans = KMeans(n_clusters=2, random_state=598).fit(lesssvddata)
print("\nKmeans:\n" + classification_report(kmeans.labels_,winner))

Logistic Regression: 83.141%
KNN: 82.777%
Decision Trees: 82.899%
Random Forest: 82.353%

Kmeans:
              precision    recall  f1-score   support

           0       0.87      0.71      0.78      2596
           1       0.74      0.89      0.81      2398

    accuracy                           0.79      4994
   macro avg       0.80      0.80      0.79      4994
weighted avg       0.81      0.79      0.79      4994



In [73]:
lessData = pd.DataFrame(sortedData, columns=labels)
lessData = lessData.drop(columns=['Score_diff', 'Opponent'])
svd = TruncatedSVD(n_components=4, n_iter=7, random_state=598)
lesssvddata = svd.fit_transform(lessData)

X_train, X_test, y_train, y_test = train_test_split(lesssvddata, winner, test_size=0.33, random_state=598)

lr = LogisticRegression(solver ='lbfgs',max_iter=100, random_state=598).fit(X_train, y_train)
print("Logistic Regression: %.3f%%" % (lr.score(X_test, y_test)*100))

neigh = KNeighborsClassifier(n_neighbors=58).fit(X_train, y_train)
print("KNN: %.3f%%" % (neigh.score(X_test, y_test)*100))


X_train, X_test, y_train, y_test = train_test_split(lessData, winner, test_size=0.33, random_state=598)

tree = DecisionTreeClassifier(max_depth=3, random_state=598).fit(X_train, y_train)
print("Decision Trees: %.3f%%" % (tree.score(X_test, y_test)*100))

forest = RandomForestClassifier(max_depth=9, random_state=598).fit(X_train, y_train)
print("Random Forest: %.3f%%" % (forest.score(X_test, y_test)*100))


kmeans = KMeans(n_clusters=2, random_state=598).fit(lesssvddata)
print("\nKmeans:\n" + classification_report(kmeans.labels_,winner))

Logistic Regression: 75.682%
KNN: 75.136%
Decision Trees: 73.681%
Random Forest: 79.563%

Kmeans:
              precision    recall  f1-score   support

           0       0.28      0.24      0.26      2542
           1       0.33      0.39      0.35      2452

    accuracy                           0.31      4994
   macro avg       0.31      0.31      0.31      4994
weighted avg       0.31      0.31      0.31      4994



In [72]:
lessData = pd.DataFrame(sortedData, columns=labels)
lessData = lessData.drop(columns=['GameType', 'Score_diff', 'Opponent'])
svd = TruncatedSVD(n_components=4, n_iter=7, random_state=598)
lesssvddata = svd.fit_transform(lessData)

X_train, X_test, y_train, y_test = train_test_split(lesssvddata, winner, test_size=0.33, random_state=598)

lr = LogisticRegression(solver ='lbfgs',max_iter=100, random_state=598).fit(X_train, y_train)
print("Logistic Regression: %.3f%%" % (lr.score(X_test, y_test)*100))

neigh = KNeighborsClassifier(n_neighbors=58).fit(X_train, y_train)
print("KNN: %.3f%%" % (neigh.score(X_test, y_test)*100))


X_train, X_test, y_train, y_test = train_test_split(lessData, winner, test_size=0.33, random_state=598)

tree = DecisionTreeClassifier(max_depth=3, random_state=598).fit(X_train, y_train)
print("Decision Trees: %.3f%%" % (tree.score(X_test, y_test)*100))

forest = RandomForestClassifier(max_depth=9, random_state=598).fit(X_train, y_train)
print("Random Forest: %.3f%%" % (forest.score(X_test, y_test)*100))


kmeans = KMeans(n_clusters=2, random_state=598).fit(lesssvddata)
print("\nKmeans:\n" + classification_report(kmeans.labels_,winner))

Logistic Regression: 75.682%
KNN: 75.136%
Decision Trees: 73.681%
Random Forest: 78.836%

Kmeans:
              precision    recall  f1-score   support

           0       0.28      0.24      0.26      2542
           1       0.33      0.39      0.35      2452

    accuracy                           0.31      4994
   macro avg       0.31      0.31      0.31      4994
weighted avg       0.31      0.31      0.31      4994

